# NCI1

In [1]:
import torch

from GNN import GCN_NCI1
from preprocessing import NCI1Dataset

## Data

In [2]:
dataset = NCI1Dataset("../../../data/NCI1")
dataset = dataset.shuffle()

In [10]:
from torch_geometric.loader import DataLoader

train_dataset = dataset[:int(0.8 * len(dataset))]
test_dataset = dataset[int(0.8 * len(dataset)):]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[866, 37], edge_index=[2, 1872], y=[32], batch=[866], ptr=[33])


## Model

In [12]:
model = GCN_NCI1(
    in_features=dataset.num_node_features,
    h_features=64,
    n_classes=2
)

In [13]:
print(model)

GCN_NCI1(
  (conv1): GraphConvolution (37 -> 64)
  (conv2): GraphConvolution (64 -> 64)
  (conv3): GraphConvolution (64 -> 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


## Train

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [15]:
best_test_acc = 0.0
for epoch in range(1, 101):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        best_model_params = model.state_dict()
        print("Checkpoint saved!")

Epoch: 001, Train Acc: 0.5703, Test Acc: 0.5633
Epoch: 002, Train Acc: 0.6098, Test Acc: 0.6277
Epoch: 003, Train Acc: 0.6016, Test Acc: 0.5985
Epoch: 004, Train Acc: 0.6293, Test Acc: 0.6156
Epoch: 005, Train Acc: 0.6390, Test Acc: 0.6241
Epoch: 006, Train Acc: 0.6223, Test Acc: 0.5827
Epoch: 007, Train Acc: 0.6429, Test Acc: 0.6642
Epoch: 008, Train Acc: 0.6734, Test Acc: 0.6642
Epoch: 009, Train Acc: 0.6667, Test Acc: 0.6375
Epoch: 010, Train Acc: 0.6837, Test Acc: 0.6898
Epoch: 011, Train Acc: 0.6886, Test Acc: 0.6934
Epoch: 012, Train Acc: 0.6788, Test Acc: 0.6569
Epoch: 013, Train Acc: 0.6883, Test Acc: 0.6873
Epoch: 014, Train Acc: 0.6901, Test Acc: 0.6655
Epoch: 015, Train Acc: 0.6810, Test Acc: 0.6618
Epoch: 016, Train Acc: 0.6898, Test Acc: 0.6679
Epoch: 017, Train Acc: 0.6898, Test Acc: 0.6642
Epoch: 018, Train Acc: 0.6764, Test Acc: 0.6655
Epoch: 019, Train Acc: 0.6907, Test Acc: 0.6618
Epoch: 020, Train Acc: 0.6816, Test Acc: 0.6740
Epoch: 021, Train Acc: 0.6849, Test Acc:

KeyboardInterrupt: 